## RAGの処理フロー

- https://zenn.dev/umi_mori/books/llm-rag-langchain-python/viewer/rag-flow

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader


loader = TextLoader("../data/work_rules.md", encoding="utf-8")
documents = loader.load()
documents

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=10,
    chunk_overlap=0,
)

docs = text_splitter.split_documents(documents)
print(docs)
print(len(docs))

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

retriever = db.as_retriever()
retriever

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

question = "基本勤務時間は、何時から、何時ですか？"
retriever.invoke(question)

In [ ]:
template = """
# ゴール
私は、就業規則の文章と質問を提供します。
あなたは、就業規則に基づいて、質問に対する回答を生成してください。

# 質問
{question}

# 就業規則
{context}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

res_1 = chain.invoke(question)
print(res_1)

## LangChain Evaluationによる評価

- https://zenn.dev/umi_mori/books/llm-rag-langchain-python/viewer/rag-accuracy-langchain-evaluation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.evaluation import load_evaluator

model = ChatOpenAI(model="gpt-4o-mini")
evaluator = load_evaluator("score_string", llm=model)

In [ ]:
question = "基本勤務時間は、何時から、何時ですか？"
reference = "基本勤務時間は、午前9時から午後6時までです。"
res_1 = "基本勤務時間は、午前9時から午後3時までです。"

eval_result_method_1 = evaluator.evaluate_strings(
    prediction=res_1,
    input=question,
    reference=reference,
)

In [ ]:
eval_result_method_1["score"]

In [ ]:
eval_result_method_1["reasoning"]